# MAPPO Integration

This runs the integrated MAPPO algorithm with the patrolling zoo.

In [4]:
%reload_ext autoreload
%autoreload 2
from onpolicy.scripts.train.train_patrolling import get_config, parse_args, main

import os
os.environ["WANDB__SERVICE_WAIT"] = "300"

In [5]:
parser = get_config()
all_args = parse_args([], parser)

all_args.experiment_name = "bitmap"
all_args.env_name = "Patrolling"
all_args.user_name = "anthony-goeckner"

all_args.num_agents = 2
all_args.agent_speed = 10.0
all_args.observe_method = "bitmap"

all_args.graph_name = "4nodes"
all_args.graph_file = f"patrolling_zoo/env/{all_args.graph_name}.graph"
all_args.num_env_steps = 10e4 #total number of steps
all_args.episode_length = 50 #number of steps in a training episode
all_args.max_cycles = 200 #number of steps in an environment episode

all_args.algorithm_name = "rmappo"
all_args.use_recurrent_policy = True
all_args.use_naive_recurrent_policy = False
all_args.use_centralized_V = True

all_args.n_rollout_threads = 25
all_args.cuda = True
all_args.cuda_index = 0

all_args.use_wandb = True

In [6]:
main([], parsed_args = all_args)

u are choosing to use rmappo, we set use_recurrent_policy to be True
choose to use gpu...



 Env Patrolling Algo rmappo Exp bitmap updates 0/80 episodes, total num timesteps 1250/100000.0, FPS 141.

average episode rewards is 28341.677856445312

 Env Patrolling Algo rmappo Exp bitmap updates 1/80 episodes, total num timesteps 2500/100000.0, FPS 152.

average episode rewards is 31816.580200195312

 Env Patrolling Algo rmappo Exp bitmap updates 2/80 episodes, total num timesteps 3750/100000.0, FPS 155.

average episode rewards is 26624.411010742188

 Env Patrolling Algo rmappo Exp bitmap updates 3/80 episodes, total num timesteps 5000/100000.0, FPS 158.

average episode rewards is 28177.301025390625

 Env Patrolling Algo rmappo Exp bitmap updates 4/80 episodes, total num timesteps 6250/100000.0, FPS 158.

average episode rewards is 29955.694580078125

 Env Patrolling Algo rmappo Exp bitmap updates 5/80 episodes, total num timesteps 7500/100000.0, FPS 158.

average episode rewards is 29325.1708984375

 Env Patrolling Algo rmappo Exp bitmap updates 6/80 episodes, total num times

actor_grad_norm,▁▁▄▂▂▂▄▂▂█▂▂▅▇▃▂▄▃▅▄▃▂▅▂▂▂▆▅▃▃▇▄▂▂▅▄▃▂▆▃
average_episode_rewards,▁▁▂▂▂▁▂▂▃▂▃▃▃▃▂▂▄▃▄▅▅▅▂▅▆▅▅▇▆▇▆▅▆▆▇▇▆█▆█
avg_idleness,▆█▇▇▇▇▆▆▅▅▅▄▅▄▅▅▄▄▄▃▃▃▅▄▂▃▃▂▂▂▂▂▁▂▁▁▁▁▂▁
critic_grad_norm,█▂▂▂▂▁▂▁▂▁▂▁▂▁▂▁▂▁▂▁▂▁▁▁▂▁▂▁▂▁▂▁▂▁▂▁▂▁▂▂
dist_entropy,██▇▇▇▇▇▇▇▆▇▆▆▆▆▆▆▆▆▅▅▅▅▄▅▄▄▃▄▃▃▂▃▂▂▁▂▁▂▁
policy_loss,▁▁▄▂▂▃▄▄▂▆▃▄▃█▃▃▄▃▃▃▄▅▂▃▃▂▃▅▄▂▆▃▄▃▁▄▃▄▆▂
ratio,▅▄▆▄▄▅▄▄▄▅▆▄▄▅▄▃▅▅▅▃▅▇▅▅▅▆█▄▄▄▃▅▃▃▄▄▁▆▃▅
value_loss,█▂▃▃▃▂▃▂▃▂▃▂▂▂▁▁▂▂▂▃▂▂▁▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▃
actor_grad_norm,0.59019
average_episode_rewards,60918.95752
avg_idleness,7.25
